# Disparity Analysis and Logistic Regression

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import random
import math
import json

import istarmap  # import to apply monkey patch to multiprocess
import multiprocess
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import stats 
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from sklearn.preprocessing import StandardScaler, RobustScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf

from aggregators import bucket_and_bin, filter_df
from config import city2ap

In [3]:
# ouputs
fn_distance = '../data/output/figs/fig2_dist.csv'
fn_disparity = '../data/output/tables/table1_disparities_by_city.csv'
fn_regression_all = '../data/output/tables/table2_regression_outputs_all.csv'
fn_regression_income = '../data/output/tables/table3a_regression_outputs_income.csv'
fn_regression_race = '../data/output/tables/table3b_regression_outputs_race.csv'
fn_regression_redlining = '../data/output/tables/table3c_regression_outputs_redlining.csv'


fn_observed_income = '../data/output/figs/fig4_income.csv'
fn_adjusted_income = '../data/output/figs/fig5_income.csv'
fn_observed_race = '../data/output/figs/fig4_race.csv'
fn_adjusted_race= '../data/output/figs/fig5_race.csv'
fn_observed_redlining = '../data/output/figs/fig4_redlining.csv'
fn_adjusted_redlining = '../data/output/figs/fig5_redlining.csv'

# inputs
fn_att = '../data/output/speed_price_att.csv.gz'
fn_centurylink = '../data/output/speed_price_centurylink.csv.gz'
fn_verizon = '../data/output/speed_price_verizon.csv.gz'
fn_earthlink = '../data/output/speed_price_earthlink.csv.gz'
inputs = {
    "AT&T" : fn_att,
    "CenturyLink": fn_centurylink,
    "Verizon": fn_verizon,
    "EarthLink" : fn_earthlink
}

# params
recalculate = False
n_jobs = 10

In [ ]:
# how many cities and offers do we have?
c = 0
cities = set()
for (isp, fn) in inputs.items():
    d = filter_df(fn, isp=isp)
    cities.update(d.major_city.unique())
    c += len(d)
    print(f"{isp} we analyzed {len(d)} addresses from {d.major_city.nunique()} cities")
c, len(cities)

In [ ]:
def which_has_largest_proportion(df, col='income_level', bins=['Low'], comparison=[], limit=300):
    sample1 = df[df[col].isin(bins)]
    if len(comparison) != 0:
        sample2 = df[df[col].isin(comparison)]
    else:
        sample2 = df[~df[col].isin(bins)]
        
    if len(sample1) < limit or len(sample2) < limit:
        return None, None, None, 'small bin'
    
    try:
        prop1 = round(len(sample1[sample1.is_slow == 1]) / len(sample1), 4)
        prop2 = round(len(sample2[sample2.is_slow == 1]) / len(sample2), 4)
        return prop1, prop2, round(prop1 - prop2, 3) >= .05, None
    except Exception as e:
        print(e, df.head(2))
        return None, None, None, e

def has_disparity(df, isp, params, limitations=False):
    """
    Checks for disparities in each city.
    """
    iv = params['iv']
    data = []
    
    s = df.groupby('major_city').race_perc_non_white.quantile([.25,.75]).unstack()
    s['delta'] = s[0.75] - s[0.25]
    race_cities = set(s[s.delta >= .2].index)
    
    for (city, state), _df in df.groupby(['major_city', 'state']):
        uniform = False
        _df = bucket_and_bin(_df, limitations=limitations)
        
        n_slow = len(_df[_df.is_slow == True])
        n_not_slow = len(_df[_df.is_slow == False])
        n_all = len(_df)
        
        variety = _df.speed_down_bins.value_counts(normalize=True)
        variety[variety >= .95]
        
        if len(variety[variety >= .95]) != 0:
            uniform = True
            
        if iv == 'income':
            a, b, is_larger, flag = which_has_largest_proportion(_df, col=params['col'], bins=params['grades'],
                                                                 comparison=params['comparison'])

        elif iv == 'redlining':
            _df_rated = _df[~_df.redlining_grade.isnull()]
            if len(_df_rated) / n_all <= .05:
                data.append({'major_city': city, 'state': state, 'isp': isp,
                             f'prop_slow_{iv}_exposure': None,
                             f'prop_slow_{iv}_treatment': None,
                             f'prop_slow_{iv}_delta' : None,
                             f'uniform_speed_{iv}': uniform,
                             f'flag_{iv}': 'not HOLC graded'})
                continue
            a, b, is_larger, flag = which_has_largest_proportion(_df, col=params['col'], bins=params['grades'],
                                                           comparison=params['comparison'])
        elif iv == 'race':
            n_minority_white = len(_df[_df.race_perc_non_white > .50])
            n_majority_white = len(_df[_df.race_perc_non_white < .50])
            # check only cities with at least 5 percent of addresses in minority white.
            if (n_minority_white / n_all <= .05) or (n_majority_white / n_all <=.05):
                data.append({'major_city': city, 'state': state, 'isp': isp,
                         f'prop_slow_{iv}_exposure': None,
                         f'prop_slow_{iv}_treatment': None,
                         f'prop_slow_{iv}_delta' : None,
                         f'uniform_speed_{iv}': uniform,
                         f'flag_{iv}' : 'no diversity'})
                continue
            a, b, is_larger, flag = which_has_largest_proportion(_df, col=params['col'], bins=params['grades'],
                                                          comparison=params['comparison'])

        data.append({
            'major_city': city,
            'state': state,
            f'slowest_{iv}' : is_larger,
            'isp': isp,
            f'prop_slow_{iv}_exposure': a,
            f'prop_slow_{iv}_treatment': b,
            f'prop_slow_{iv}_delta' : a-b if a else None,
            f'uniform_speed_{iv}': uniform,
            f'flag_{iv}': flag
        })
        
    return data

In [ ]:
# renaming columns, because I am not as smart as I think I am...
col2colrename = {
    'prop_slow_income_exposure': 'pct_slow_lower_income',
    'prop_slow_income_treatment': 'pct_slow_upper_income',
    'prop_slow_income_delta' : 'income_pct_pt_diff',
    'prop_slow_race_exposure': 'pct_slow_least_white',
    'prop_slow_race_treatment': 'pct_slow_most_white',
    'prop_slow_race_delta' : 'race_pct_pt_diff',
    'prop_slow_redlining_exposure': 'pct_slow_d_rated',
    'prop_slow_redlining_treatment': 'pct_slow_ab_rated',
    'prop_slow_redlining_delta' : 'redlining_pct_pt_diff',
    'uniform_speed_income': 'uniform_speed',
    'slowest_income': 'income_disparity',
    'slowest_race': 'race_disparity',
    'slowest_redlining': 'redlining_disparity'
}

back2colname = {v:k for k,v in col2colrename.items()}

In [ ]:
income_params = {'iv': "income", 'grades': ['Low'], 'col': 'income_level'}
race_params = {'iv': "race", 'grades': ['least white'], 'col': 'race_quantile'}
redlining_params = {'iv': "redlining", 'grades': ['D'], 'col': 'redlining_grade'}

# Treatment group
income_params['comparison'] = ['Upper Income']
race_params['comparison'] = ['most white']
redlining_params['comparison'] = ['A', 'B']

In [ ]:
limitations = True # set to True for different categorization criteria for income and race/ethnicity

if not os.path.exists(fn_disparity) or recalculate:
    table = pd.DataFrame()
    for isp, fn in tqdm(inputs.items()):
        df = filter_df(fn, isp)
        income_data = has_disparity(df, isp=isp, params=income_params, limitations=limitations)
        race_data = has_disparity(df, isp=isp, params=race_params, limitations=limitations)
        redlining_data = has_disparity(df, isp=isp, params=redlining_params, limitations=limitations)

        df_table = pd.DataFrame(income_data).merge(
            pd.DataFrame(race_data), on=['major_city', 'isp', 'state'], how='outer'
        ).merge(
            pd.DataFrame(redlining_data), on=['major_city', 'isp', 'state'], how='outer'
        )

        table = table.append(df_table)
    table.major_city = table.major_city.str.title()
    table.columns = [col2colrename.get(c, c) for c in table.columns]
    table = table[[c for c in table.columns if c not in ['uniform_speed_race', 'uniform_speed_redlining']]]
    if not limitations:
        table.to_csv(fn_disparity, index=False)

else:
    table = pd.read_csv(fn_disparity)

In [ ]:
# all cities in the study
n_cities = table.major_city.nunique()
n_cities
trend = set(table[(table.income_disparity == True) | 
           (table.race_disparity == True) | 
           (table.redlining_disparity == True)].major_city.unique())

In [ ]:
# how many cities exhibit none of these trends?
set(table[table.uniform_speed == False].major_city.unique()) - trend

In [ ]:
# How many cities have a redlining disparity that is 2x or greater?
table['x_red'] = table.pct_slow_d_rated.div(table.pct_slow_ab_rated)
redlined_2x_often = table[(table.x_red >= 2) & (table.uniform_speed == False)]

redlined_2x_often.major_city.unique(), redlined_2x_often.major_city.nunique()

In [ ]:
for params in [income_params, race_params ,redlining_params]:
    iv = params['iv']
    base_group = params['grades'][0]
    
    # filter out uniform cities here
    table_ = table[table[f"uniform_speed"] == False]

    # filter out cases we omitted for lack of data
    null = table_[table_[f'{iv}_disparity'].isnull()]
    null_city =  (set(table_[table_[f'{iv}_disparity'].isnull()].major_city.unique()) -
                     set(table_[~table_[f'{iv}_disparity'].isnull()].major_city))
    testable = table_[~table_[f'{iv}_disparity'].isnull()]
    all_cities_tested = testable.major_city.nunique()
    
    # of significant cases, when is the slowest class the lower income/least white/redlined example
    slowest = testable[testable[f"{iv}_disparity"] == True]
    perc_slower= len(slowest)
    perc_slower_city = slowest.major_city.nunique()
    print(f"{iv.title()} {len(testable)} city-ISP pairs and {all_cities_tested} cities"
          f"\n - {base_group} areas disproportionately offered slow speeds {perc_slower} ({perc_slower/len(testable)*100:.1f}%) city-isp pairs and {perc_slower_city} ({perc_slower_city/all_cities_tested*100:.1f}%) cities."
          f"\n - omit {len(null)} city-isp-pairs and {len(null_city)} cities. \n")

In [ ]:
random.seed(303)
for isp in inputs.keys():
    table_ = table[table.isp == isp]
    if isp == 'EarthLink':
        table_['major_city'] = table_.apply(lambda x: f"{x['major_city']} ({x['state'].upper()})", axis=1)
        
    null_cities = table_[table_.uniform_speed == True].major_city.unique()
    table_ = table_[table_.uniform_speed == False]
    all_cities = table_.major_city.nunique()
    all_cities -= len(null_cities)

    # income
    n_cities = table_[~table_.income_disparity.isnull()].major_city.nunique()    
    display(Markdown(f"{isp} (N={n_cities}) disporportionate disparity of slow speeds in..."))

    yes_disparity = table_[table_.income_disparity== True].major_city.unique()
    no_disparity = table_[table_.income_pct_pt_diff.between(-.05, .05, inclusive="neither")].major_city.unique()
    opposite_disparity = table_[table_.income_pct_pt_diff <= -.05].major_city.unique()
    
    random.shuffle(yes_disparity)
    n_low = len(yes_disparity)

    text =  (
        f"- {' and '.join(income_params['grades'])} Income areas than Upper Income areas in **{round(n_low / n_cities * 100, 1)}**% of cities ({n_low})."
    )
    if len(yes_disparity) > 1:
        cities_ = ', '.join(yes_disparity[:3])
        text += f"<br>This includes cities such as: {cities_}."
    if len(no_disparity) > 0:
        perc = round(len(no_disparity) / n_cities * 100, 1)
        text += f"<br>No disparities in {perc}% of cities ({len(no_disparity)})."
    if len(opposite_disparity) > 0:
        perc = round(len(opposite_disparity) / n_cities * 100, 1)
        text += f"<br>The opposite trend was in {perc}% of cities ({len(opposite_disparity)})."
    display(Markdown(text))

    # race
    n_cities = table_[~table_.race_disparity.isnull()].major_city.nunique()
    cities = table_[table_.race_disparity == True].major_city.unique()
    yes_disparity = table_[table_.race_disparity == True].major_city.unique()
    no_disparity = table_[table_.race_pct_pt_diff.between(-.05, .05, 
                                                          inclusive="neither")].major_city.unique()
    opposite_disparity = table_[table_.race_pct_pt_diff <= -.05].major_city.unique()
    random.shuffle(yes_disparity)
    n_minority_white = len(yes_disparity)
    
    text = (
        f"- the {' and '.join(race_params['grades'])} areas compared to the most white areas in <b>{round(n_minority_white / n_cities * 100, 1)}</b>% of cities ({n_minority_white})."
    )
    if len(yes_disparity) > 1:
        cities_ = ', '.join(yes_disparity[:3])
        text += f"<br>This includes cities such as: {cities_}."
    if len(no_disparity) > 0:
        perc = round(len(no_disparity) / n_cities * 100, 1)
        text += f"<br>No disparities in {perc}% of cities ({len(no_disparity)})."
    if len(opposite_disparity) > 0:
        perc = round(len(opposite_disparitiy) / n_cities * 100, 1)
        text += f"<br>The opposite trend was in {perc}% of cities ({len(opposite_disparitiy)})."
 
    
    if n_cities < all_cities:
        diff = all_cities - n_cities
        text += f"<br>We exclude {diff} {'cities' if diff > 1 else 'city'} for not containing enough minority white areas for comparison."
    display(Markdown(text))

    # redlining
    n_cities = table_[~table_.redlining_disparity.isnull()].major_city.nunique()
    yes_disparity = table_[table_.redlining_disparity == True].major_city.unique()
    no_disparity = table_[table_.redlining_pct_pt_diff.between(-.05, .05,
                                                               inclusive="neither")].major_city.unique()
    opposite_disparity = table_[table_.redlining_pct_pt_diff <= -.05].major_city.unique()
    random.shuffle(cities)
    n_redline = len(cities)

    text = (
        f"- {' and '.join(redlining_params['grades'])}"
        f" graded areas than A and B graded areas in **{round(n_redline/ n_cities * 100, 1)}**% of cities ({n_redline})."
    )
    if len(yes_disparity) > 1:
        cities_ = ', '.join(yes_disparity[:3])
        text += f"<br>This includes cities such as: {cities_}."
    if len(no_disparity) > 0:
        perc = round(len(no_disparity) / n_cities * 100, 1)
        text += f"<br>No disparities in {perc}% of cities ({len(no_disparity)})."
    if len(opposite_disparity) > 0:
        perc = round(len(opposite_disparity) / n_cities * 100, 1)
        text += f"<br>The opposite trend was statistically significant in {perc}% of cities ({len(opposite_disparity)})."
    
    if n_cities < all_cities:
        diff = all_cities - n_cities
        text += f"<br>We exclude {diff} {'cities' if diff > 1 else 'city'} for not containing enough HOLC graded areas for comparison."
    display(Markdown(text))

    if len(null_cities) > 0:
        n_null = len(null_cities)
        if n_null == 1:
            text = f"Note: {null_cities[0].title()} was only offered slow speeds. We exclude this city from the above calculations."
        else:
            text =(
                f"Note: we exclude {n_null} {'cities' if n_null > 1 else 'city'} that are served the same speed."
                f" This includes {', '.join([c.title() for c in null_cities[:3]])}."
            )
        display(Markdown(text))
    display(Markdown('<hr>'))

### What patterns emerge when we collapose the ISPs together, and look at cities?

In [ ]:
table.major_city.nunique()

In [ ]:
income_cities = table[(~table.income_disparity.isnull()) & (table.uniform_speed == False)].major_city.unique()
race_cities = table[(~table.race_disparity.isnull()) & (table.uniform_speed == False)].major_city.unique()
redlining_cities = table[(~table.redlining_disparity.isnull()) & (table.uniform_speed == False)].major_city.unique()

all_cities = table[table.uniform_speed == False].major_city.nunique()

In [ ]:
color_1 = '#4f0d70'
color_2 = "#C31B6A"

bool2alpha = {
    True: .95,
    False : .35
}

In [ ]:
def asterix_city(row, iv='income'):
    if row[f'flag_{iv}'] == 'monospeed' or row[f'uniform_speed'] == True:
        return city2ap.get(row['major_city'], row['major_city']) + '*'
    if row[f'flag_{iv}'] in {'not HOLC graded', 'small bin',  'no diversity'}:
        return city2ap.get(row['major_city'],row['major_city'])  + '˟'
    return city2ap.get(row['major_city'], row['major_city'])

def plot_observed(to_plot, iv='income', fn=None, title=None, ylim=60.5):    
    to_plot['Observed'] = to_plot[f'{iv}_pct_pt_diff']
    to_plot_ = to_plot[~to_plot['Observed'].isnull()].copy()
#     to_plot_ = to_plot.copy()
    to_plot_.sort_values(by=['isp', 'Observed'], ascending=[True, False], inplace=True)
    to_plot_['display_city'] = to_plot_.apply(asterix_city, axis=1, iv=iv)
    
    cols = [
        'Observed', 
        col2colrename.get(f'prop_slow_{iv}_exposure'), 
        col2colrename.get(f'prop_slow_{iv}_treatment')
    ]
    for col in cols:
        to_plot_[col] = to_plot_[col] * 100
    
    to_plot_['major_city_isp'] = to_plot_.apply(lambda x: x['isp'] + ' - ' + x['display_city'], axis=1)

    ax = to_plot_.plot(y='major_city_isp', x='Observed', 
                      color=color_1,
                      kind='scatter', figsize=(6, 13))

    to_plot_['isp'] = pd.Categorical(to_plot_['isp'], ["AT&T", "Verizon", "CenturyLink", "EarthLink"])
    to_plot_ = to_plot_.sort_values(by=['isp', 'Observed'], 
                                    ascending=[True, False])
    cols.extend(['isp', 'major_city', 'display_city'])
    to_plot_[cols].to_csv(fn, index=False)
    
    to_plot_ = to_plot[(~to_plot['Observed'].isnull()) & (to_plot['uniform_speed'] == False)].copy()


    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)

    ax.spines['top'].set_visible(False)

    # Only show ticks on the left and bottom spines
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    plt.gca().invert_yaxis()

    ax.set_ylabel("")
    ax.set_xlabel("Percentage point difference")

    plt.axvline(x=5,linewidth=1, color='black', linestyle='--', ymax=.96)
    plt.axvline(x=-5,linewidth=1, color='black', linestyle='--', ymax=.96)

    plt.text(-8, -3., 
             title,
             size=14,
             horizontalalignment='left',
             verticalalignment='center',);
    ax.set_ylim(bottom=ylim)


In [ ]:
plot_observed(table, iv='income', fn=fn_observed_income,
              ylim=57,
              title = "Compared to Upper Income areas,\nLow income areas were disproportionately\noffered slow speeds in almost every city",)

In [ ]:
plot_observed(table, iv='race', fn=fn_observed_race, ylim=50,
              title = f"Compared to Whiter counterparts,\nthe Least White Areas were Disproportionately\nOffered Slow Speeds in Almost Every City")

In [ ]:
plot_observed(table, iv='redlining', fn=fn_observed_redlining, ylim=34,
              title = f"Compared to better HOLC-rated areas,\nHistorically Redlined Areas were Disproportionately\nOffered Slow Speeds in Every City",
)

## Regression

In [ ]:
def preprocess_for_log_reg(df):
    df['n_providers'] = df['n_providers'] - 1
    df['income_dollars_below_median'] = df['income_dollars_below_median'] / 100

    df['constant'] = 1
    scaler = StandardScaler()
    df['lat_standard'] = scaler.fit_transform(df[['lat']])[:, 0]
    scaler = StandardScaler()
    df['lon_standard'] = scaler.fit_transform(df[['lon']])[:, 0]
    scaler = StandardScaler()
    df['ppl_per_sq_mile_standard'] = scaler.fit_transform(df[['ppl_per_sq_mile']])[:, 0]
    scaler = StandardScaler()
    df['income_dollars_below_median'] = scaler.fit_transform(df[['income_dollars_below_median']])[:, 0]
    
    # if there are no competitors, make this variable null.
    # Then this variable is linked to the outcome.
    df.loc[df.n_providers < 1, 'internet_perc_broadband'] = None
    
    scaler = StandardScaler()
    df['internet_perc_broadband'] = scaler.fit_transform(df[['internet_perc_broadband']])[:, 0]

    # this is our DV
    df['is_slow'] = df.apply(
        lambda x: 1 if x['speed_down_bins'] == "Slow (<25 Mbps)" else 0, 
        axis=1
    )
    
    for iv in ['income_level', 'race_quantile', 'redlining_grade']:
        df[iv] = df[iv].apply(lambda x: grade2rest.get(x, x))
    return df  

In [ ]:
def train_and_return_coefs(formula, df, city, isp, model_name):
    """
    Workflow for fitting binary logistic regression for `formula`.
    This is formatted strangely because it'll be called by multiprocess(ing).Pool.
    """
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
    import pandas as pd
    import numpy as np
    try:
        results = smf.logit(formula=formula, data=df).fit(disp=0)
        coefs = pd.DataFrame({
            'coef': results.params.values,
            'odds_ratio': np.exp(results.params.values),
            'pvalue': results.pvalues,
            'pr_sq': results.prsquared,
            'N' : len(df),
            'major_city': city,
            'state' : df.state.iloc[0],
            'isp': isp,
            'model' : model_name,
            'intercept': results.params['Intercept'],
        })
    except Exception as e:
        coefs = pd.DataFrame([{
            'N' : len(df),
            'major_city': city,
            'state' : df.state.iloc[0],
            'isp': isp,
            'model' : model_name,
            'error': e
        }])
    
    return coefs

In [ ]:
def odds_to_probability(row, step=1):
    """
    See about this equation here:
    http://faculty.cas.usf.edu/mbrannick/regression/Logistic.html
    """
    a = row['intercept']
    b = row['coef']
    X = step
    try:
        return math.exp(a + b * X) / (1 + math.exp(a + b * X))
    except:
        return None

In [ ]:
iv2treatment = {
    "income_level" : 'Upper Income',
    "race_quantile" : 'most white',
    "redlining_grade" : 'rest'
}

grade2rest = {
    'A' : 'rest',
    'B' : 'rest',
}

In [ ]:
if not os.path.exists(fn_regression_all) or recalculate:
    data_regression = pd.DataFrame([])
    
    args = []
    for isp, fn in inputs.items():
        df = filter_df(fn, isp)
        df['major_city'] = df['major_city'].apply(lambda x: x.title())
        for (city, state), _df in df.groupby(by=['major_city', 'state']):
            _df = bucket_and_bin(_df)
            _df = preprocess_for_log_reg(_df)
            
            n_slow = len(_df[_df.speed_down_bins == 'Slow (<25 Mbps)'])
            n_not_slow = len(_df[_df.speed_down_bins != 'Slow (<25 Mbps)'])
            n_all = len(_df)
            if (_df.speed_down_bins.nunique() <= 1) or (n_slow / n_all <= .01) or (n_not_slow / n_all <= .01):
                # not enough speed
                continue
            for iv, treatment in iv2treatment.items():
                if iv == 'race_quantile':
                    n_minority_white = len(_df[_df.race_perc_non_white > .5])    
                    n_majority_white = len(_df[_df.race_perc_non_white < .5])            

                    # check only cities with at least 5 percent of addresses in minority white.
                    if n_minority_white / len(_df) <= .05 or n_majority_white / len(_df) <= .05:
                        print(f"skip {city} {isp} race")
                        continue
                elif iv == 'redlining_grade':
                    _df_rated = _df[~_df.redlining_grade.isnull()]
                    if len(_df_rated) <= len(_df) * .05:
                        print(f"skip {city} {isp} redline")
                        continue
                    if 'D' not in _df['redlining_grade'].unique().tolist():
                        print(f"skip {city} {isp} redline")
                        continue

                formula = f"is_slow ~ C({iv}, Treatment('{treatment}'))"
                args.append([formula, _df, city, isp, f"{iv}_alone"])
                if city in ['Omaha', 'Phoenix'] and iv == 'redlining_grade':
                    formula += " + ppl_per_sq_mile_standard + internet_perc_broadband"
                else:
                    formula += " + ppl_per_sq_mile_standard + n_providers + internet_perc_broadband"
                args.append([formula, _df, city, isp, f"{iv}_controls"])

                # ablation formulas
                if city in ['Omaha', 'Phoenix'] and iv == 'redlining_grade':
                    formulas = [" + ppl_per_sq_mile_standard", " + internet_perc_broadband"]
                else:
                    formulas = [" + ppl_per_sq_mile_standard", " + n_providers ",  " + internet_perc_broadband"]
                for iv_ in formulas:
                    formula = f"is_slow ~ C({iv}, Treatment('{treatment}'))"
                    formula += "".join([f for f in formulas if f != iv_])
                    model_name =  f"{iv}_minus{iv_.replace(' + ', '_')}"
                    args.append([formula, _df, city, isp, model_name])

    with multiprocess.get_context("spawn").Pool(n_jobs) as pool:
        for coefs in tqdm(pool.istarmap(train_and_return_coefs, args), 
                           total=len(args)):
            data_regression = data_regression.append(coefs)

    # treatment group IE upper income, most white, best graded
    data_regression['probability_treatment'] = data_regression.apply(odds_to_probability, step=0, axis=1)
    # exposure group IE lower income, least white, redlined
    data_regression['probability_exposure'] = data_regression.apply(odds_to_probability, step=1, axis=1)
    data_regression['odds_directional'] = data_regression.apply(lambda x: x['odds_ratio'] * -1 if x['coef'] <0 else  x['odds_ratio'], axis=1)
    data_regression['is_coef_positive'] = data_regression.coef > 0
    # positive if exposure group has higher liklihood of slower speeds than treatment group
    data_regression['probability_delta'] = data_regression['probability_exposure'] - data_regression['probability_treatment']
    data_regression['perc_probability'] =  data_regression['probability_exposure'] / data_regression['probability_treatment']
    data_regression = data_regression[data_regression.index != 'Intercept']

    data_regression.to_csv(fn_regression_all)
    controls = data_regression[data_regression.model.str.contains('_controls')]
    controls.loc["C(income_level, Treatment('Upper Income'))[T.Low]"].to_csv(fn_regression_income)
    controls.loc["C(race_quantile, Treatment('most white'))[T.least white]"].to_csv(fn_regression_race)
    controls.loc["C(redlining_grade, Treatment('rest'))[T.D]"].to_csv(fn_regression_redlining)
    
else:
    data_regression = pd.read_csv(fn_regression_all, index_col=0)

## Ablation

In [ ]:
# mean psuedo r-squared (goodness of fit) for each model across cities and ISPs.
for model, _df in data_regression.groupby(['model']):
    mean_pr = _df.groupby(['major_city', 'state', 'isp']).first().pr_sq.mean()
    print(model, mean_pr)

The ablated IV that results in the least change from the observed are the most influential.

In [ ]:
income_ablation = data_regression[
    (data_regression['model'].str.contains('income_level_minus_')) &
    (data_regression.index == "C(income_level, Treatment('Upper Income'))[T.Low]") &
    (data_regression.major_city.isin(income_cities))
].merge(table, on=['isp', 'major_city', 'state'])
income_ablation['delta'] = abs(income_ablation['probability_delta'] - income_ablation['income_pct_pt_diff'])
income_ablation.sort_values(by='delta', ascending=True).groupby(['major_city', 'isp']).first().model.value_counts(normalize=True)

In [ ]:
race_ablation = data_regression[
    (data_regression['model'].str.contains('race_quantile_minus_')) &
    (data_regression.index == "C(race_quantile, Treatment('most white'))[T.least white]") &
    (data_regression.major_city.isin(race_cities))
].merge(table, on=['isp', 'major_city', 'state'])
race_ablation['delta'] = abs(race_ablation['probability_delta'] - race_ablation['race_pct_pt_diff'])
race_ablation.sort_values(by='delta', ascending=True).groupby(['major_city', 'isp']).first().model.value_counts(normalize=True)

In [ ]:
redlining_ablation = data_regression[
    (data_regression['model'].str.contains('redlining_grade_minus_')) &
    (data_regression.index == "C(redlining_grade, Treatment('rest'))[T.D]") &
    (data_regression.major_city.isin(redlining_cities))
].merge(table, on=['isp', 'major_city', 'state'])
redlining_ablation['delta'] = abs(redlining_ablation['probability_delta'] - redlining_ablation['redlining_pct_pt_diff'])
redlining_ablation.sort_values(by='delta', 
                               ascending=False).groupby(['major_city', 'isp']).first().model.value_counts(normalize=True)

## Where to disparities persist?

In [ ]:
income = (
    data_regression[
        (data_regression.model == f'income_level_controls') &
        (data_regression.index == "C(income_level, Treatment('Upper Income'))[T.Low]")
    ].sort_values('coef', ascending=False)
    .merge(
        data_regression[
            (data_regression.model == f'income_level_alone') &
            (data_regression.index == "C(income_level, Treatment('Upper Income'))[T.Low]")
        ], how='left', on=['major_city', 'state', 'isp'],
        suffixes=['', '_alone']
    )
).drop_duplicates(subset=['isp', 'major_city', 'state'])
income = income[income.major_city.isin(income_cities)]

race = (
    data_regression[
        (data_regression.model == f'race_quantile_controls') &
        (data_regression.index == "C(race_quantile, Treatment('most white'))[T.least white]")
    ].sort_values(['coef', 'state'], ascending=False)
    .merge(
        data_regression[
            (data_regression.model == f'race_quantile_alone') &
            (data_regression.index == "C(race_quantile, Treatment('most white'))[T.least white]")
        ], how='left', on=['major_city', 'state', 'isp'],
        suffixes=['', '_alone']
    )
).drop_duplicates(subset=['isp', 'major_city', 'state'])
race = race[race.major_city.isin(race_cities)]

redlining = (
    data_regression[
        (data_regression.model == f'redlining_grade_controls') &
        (data_regression.index == "C(redlining_grade, Treatment('rest'))[T.D]")
    ].sort_values('coef', ascending=False)
    .merge(
        data_regression[
            (data_regression.model == f'redlining_grade_alone') &
            (data_regression.index == "C(redlining_grade, Treatment('rest'))[T.D]")
        ], how='left', on=['major_city', 'state', 'isp'],
        suffixes=['', '_alone']
    )
).drop_duplicates(subset=['isp', 'major_city', 'state'])
redlining = redlining[redlining.major_city.isin(redlining_cities)]

In [ ]:
city_isp_income = income[~income.pvalue.isnull()]
slowest_income = city_isp_income[city_isp_income.probability_delta > 0.05]

city_isp_race = race[~race.pvalue.isnull()]
slowest_race = city_isp_race[city_isp_race.probability_delta > 0.05]

city_isp_redlining = redlining[~redlining.pvalue.isnull()]
slowest_redlining = city_isp_redlining[city_isp_redlining.probability_delta > 0.05]

print(f"Income ({len(city_isp_income)} city-ISP pairs and {city_isp_income.major_city.nunique()} cities) even after adjusting for other factors..."
     f"\n- lower income areas recieve greater proprotion of slow speeds in {len(slowest_income)} city-ISP pairs, or {len(slowest_income) / len(city_isp_income) * 100:.1f}% of cases.\n"
     f"- same pattern in {slowest_income.major_city.nunique()} out of {city_isp_income.major_city.nunique()} cities ({slowest_income.major_city.nunique() / city_isp_income.major_city.nunique():.2f}%)")
print(f"Race and Ethnicity ({len(city_isp_race)} city-ISP pairs) even after adjusting for other factors..."
     f"\n- least white areas recieve greater proportion of slow speeds in {len(slowest_race)} city-ISP pairs, or {len(slowest_race) / len(city_isp_race) * 100:.1f}% of cases.\n"
     f"- same pattern in {slowest_race.major_city.nunique()} out of {city_isp_race.major_city.nunique()} cities ({slowest_race.major_city.nunique() / city_isp_race.major_city.nunique():.2f}%)")

print(f"Redlining ({len(city_isp_redlining)} city-ISP pairs) even after adjusting for other factors..."
     f"\n- hazardous areas recieve greater proprotion of slow speeds in {len(slowest_redlining)} city-ISP pairs, or {len(slowest_redlining) / len(city_isp_redlining) * 100:.1f}% of cases.\n"
     f"- same pattern in {slowest_redlining.major_city.nunique()} out of {city_isp_redlining.major_city.nunique()} cities ({slowest_redlining.major_city.nunique() / city_isp_redlining.major_city.nunique():.2f}%)")


In [ ]:
# what percentage of AT&T cities do income disparities persist?
income_att = income[income.isp == 'AT&T']
len(income_att[income_att.probability_delta >= .05]) / len(income_att)

Misc model outputs for methodology

In [ ]:
len(income[income.pvalue < .05]) / len(income)

In [ ]:
income.pr_sq.describe()

In [ ]:
income.odds_ratio.apply(lambda x: x>1.5).value_counts(normalize=True)

In [ ]:
len(race[race.pvalue < .05]) / len(race)

In [ ]:
race.pr_sq.describe()

In [ ]:
race.odds_ratio.apply(lambda x: x>1.5).value_counts(normalize=True)

In [ ]:
len(redlining[redlining.pvalue < .05]) / len(redlining)

In [ ]:
redlining.pr_sq.describe()

In [ ]:
redlining.odds_ratio.apply(lambda x: x>1.5).value_counts(normalize=True)

### See the factor that lead to the greatest adjustments for any city

In [ ]:
income_ablation_ = data_regression[
    (data_regression['major_city'] == 'Seattle') &
    (data_regression['model'].str.contains('income_level_minus_')) & 
    (data_regression.index == "C(income_level, Treatment('Upper Income'))[T.Low]")
].merge(table, on=['isp', 'major_city', 'state'])
income_ablation_['delta'] = abs(income_ablation_['probability_delta'] - 
                                income_ablation_['income_pct_pt_diff'])
income_ablation_.sort_values(by='delta', ascending=True)

In [ ]:
race_ablation_ = data_regression[
    (data_regression['major_city'] == 'Charleston') &
    (data_regression['model'].str.contains('race_quantile_minus_')) &
    (data_regression.index == "C(race_quantile, Treatment('most white'))[T.least white]")
].merge(table, on=['isp', 'major_city', 'state'])
race_ablation_['delta'] = abs(race_ablation_['probability_delta'] - 
                              race_ablation_['race_pct_pt_diff'])
race_ablation_.sort_values(by='delta', ascending=True)

In [ ]:
redlining_ablation_ = data_regression[
    (data_regression['major_city'] == 'Minneapolis') &
    (data_regression['model'].str.contains('redlining_grade_minus_')) &
    (data_regression.index == "C(redlining_grade, Treatment('rest'))[T.D]")
].merge(table, on=['isp', 'major_city', 'state'])
redlining_ablation_['delta'] = abs(redlining_ablation_['probability_delta'] - 
                                  redlining_ablation_['redlining_pct_pt_diff'])
redlining_ablation_.sort_values(by='delta', ascending=True)

## Plots with Adjustments

In [ ]:
label2marker = {
    'Observed': 'o',
    'Adjusted': '^'
}

In [ ]:
def plot_adjusted(to_plot, ylim=33.5, figsize=(6, 8), fn=fn_adjusted_redlining,
                  title= f"After adjusting for other factors, Redlined areas\nstill disprportionately offered slow Internet speeds",
                  title_y = -3.2, title_x= -.05):
    legend_elements = [Line2D([0], [0], marker=label2marker.get(label), color='w', 
                          label=label, markerfacecolor=c, markersize=10)
                   for label, c in {'Observed': color_1, 'Adjusted': color_2}.items()][::-1] 
    
    to_plot_ = to_plot#[(to_plot['pvalue'] < 0.05)]
    to_plot_['major_city_isp'] = to_plot_.apply(lambda x: x['isp'] + ' - ' + x['major_city'], axis=1)

    to_plot_['Observed'] = to_plot_.probability_delta_alone * 100
    to_plot_['Adjusted'] = to_plot_.probability_delta * 100

    to_plot_ = to_plot_.sort_values(by=['Adjusted', 'state', 'Observed'], ascending=False)

    ax = to_plot_.plot(y='major_city_isp', x='Observed', 
                      color=color_1,
                      kind='scatter', figsize=figsize)
    to_plot_.plot(y='major_city_isp', x='Adjusted', 
                 color=color_2, ax=ax, marker='^',
                 alpha = [bool2alpha.get(_) for _ in to_plot_.pvalue < 0.05],
                 kind='scatter')

    x1 = to_plot_['Adjusted'].tolist()
    x2 = to_plot_['Observed'].tolist()
    n = to_plot_['major_city_isp'].tolist()
    for i in range(0, len(x1)):
        ax.plot([x1[i], x2[i]], [n[i], n[i]], '-', 
                color='black', zorder=-1, alpha=.8)

    to_plot_['isp'] = pd.Categorical(to_plot_['isp'], ["AT&T", "Verizon", "CenturyLink", "EarthLink"])
    to_plot_ = to_plot_.sort_values(by=['isp','Adjusted', 'state', 'Observed'], 
                                    ascending=[True, False, False, False])
    to_plot_[['major_city', 'isp', 'Adjusted', 'Observed', 'state']].to_csv(fn, index=False)    

    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)

    ax.spines['top'].set_visible(False)

    # Only show ticks on the left and bottom spines
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    plt.gca().invert_yaxis()

    ax.set_ylabel("")
    ax.set_xlabel("Percentage point difference")

    plt.axvline(x=0,linewidth=1, color='black', linestyle='--', ymax=.96)

    plt.text(title_x, title_y, 
             title,
             size=14,
             horizontalalignment='left',
             verticalalignment='center',);
    ax.set_ylim(bottom=ylim)

    ax.legend(handles=legend_elements[::-1],
                  loc='lower left', 
                  bbox_to_anchor= (-0.025, .97), 
                  ncol=5,
                  handletextpad=0.0,
                  labelspacing=0, 
                  borderaxespad=.1, 
                  borderpad=0.1,
                  frameon=False,
                  prop={'size': 9.2})

In [ ]:
to_plot = income
plot_adjusted(to_plot, ylim=58.5, figsize=(6, 10), 
              fn=fn_adjusted_income,
              title_y = -4.2, title_x = -14, 
              title= f"After adjusting for other factors, low income areas\nstill disporportionately offered slow Internet speeds",
)

In [ ]:
to_plot = race
plot_adjusted(to_plot, ylim=52.5, figsize=(6, 10), 
              fn=fn_adjusted_race,
              title_y = -3.5, title_x = -15,
              title= f"After adjusting for other factors, least white areas\nstill disproportionately offered slow Internet speeds",
)

In [ ]:
to_plot = redlining
plot_adjusted(to_plot, title_y=-3.2, title_x=-6, ylim=37.5, fn=fn_adjusted_redlining)

### Rates per plan

In [ ]:
fn_rates = '../data/output/figs/story_rates.csv'

In [ ]:
isp_rates = []
for isp, fn in inputs.items():
    df_ = filter_df(fn, isp)
    # remove no service and affordable plans
    df_ = df_[(df_.speed_down != 0) & (df_.price > 30)]
    df_['std_rate'] = df_['price'] / df_['speed_down']
    rate_lowest = df_['std_rate'].min()
    rate_highest = df_['std_rate'].max()
    
    isp_rates.append({
        'isp' : isp,
        'rate_lowest': rate_lowest,
        'rate_highest': rate_highest,
        'price' : df_['price'].iloc[0]
    })

In [ ]:
isp_rates

In [ ]:
pd.DataFrame(isp_rates).to_csv(fn_rates, index=False)